# Memory
Store memory of prior conversations. Some options with LangChain:
- Store all messages
- Store a limited number of interactions
- Store a limited number of tokens

It also comes with specialized chains that store memory with other transformations, such as a vector store or auto-summarizing conversations.

Topics:
- ChatMessageHistory
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationSummaryMemory

# ChatMessageHistory

In [1]:
from langchain.memory import ChatMessageHistory

In [2]:
history = ChatMessageHistory()

In [3]:
history.add_user_message("Hello, nice to meet you")
history.add_ai_message("Nice to meet you!")

In [4]:
history.messages

[HumanMessage(content='Hello, nice to meet you', additional_kwargs={}, example=False),
 AIMessage(content='Nice to meet you!', additional_kwargs={}, example=False)]

# ConversationBufferMemory

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [17]:
llm = ChatOpenAI()

In [18]:
memory = ConversationBufferMemory()

In [19]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

Contrary to previous cases, we now don't need to store the output into a variable, they are stored in memory.

In [20]:
conversation.predict(input="Hello, nice to meet you")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, nice to meet you
AI:

> Finished chain.


"Hello! It's a pleasure to meet you too. I'm an AI designed to assist with various tasks and provide information. How can I help you today?"

In [21]:
conversation.predict(input="Tell me about an interesting Physics fact.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, nice to meet you
AI: Hello! It's a pleasure to meet you too. I'm an AI designed to assist with various tasks and provide information. How can I help you today?
Human: Tell me about an interesting Physics fact.
AI:

> Finished chain.


"Sure! Here's an interesting Physics fact for you: Did you know that light travels at approximately 299,792,458 meters per second in a vacuum? This is known as the speed of light, and it is the fastest speed that anything can travel in the universe. It plays a crucial role in various phenomena, such as determining the behavior of electromagnetic waves and the principles of relativity."

In [25]:
print(memory.buffer)

Human: Hello, nice to meet you
AI: Hello! It's a pleasure to meet you too. I'm an AI designed to assist with various tasks and provide information. How can I help you today?
Human: Tell me about an interesting Physics fact.
AI: Sure! Here's an interesting Physics fact for you: Did you know that light travels at approximately 299,792,458 meters per second in a vacuum? This is known as the speed of light, and it is the fastest speed that anything can travel in the universe. It plays a crucial role in various phenomena, such as determining the behavior of electromagnetic waves and the principles of relativity.


In [26]:
memory.load_memory_variables({})

{'history': "Human: Hello, nice to meet you\nAI: Hello! It's a pleasure to meet you too. I'm an AI designed to assist with various tasks and provide information. How can I help you today?\nHuman: Tell me about an interesting Physics fact.\nAI: Sure! Here's an interesting Physics fact for you: Did you know that light travels at approximately 299,792,458 meters per second in a vacuum? This is known as the speed of light, and it is the fastest speed that anything can travel in the universe. It plays a crucial role in various phenomena, such as determining the behavior of electromagnetic waves and the principles of relativity."}

## How to store this into a file?

Easiest way, with pickle to write to disk.

In [28]:
# conversation.memory

In [29]:
import pickle

In [30]:
pickled_str = pickle.dumps(conversation.memory)

In [31]:
# wb: write bytes
with open("conversation_memory.pkl", mode="wb") as f:
    f.write(pickled_str)

In [37]:
# rb: read bytes
with open("conversation_memory.pkl", mode="rb") as f:
    new_memory_loaded = f.read()

llm = ChatOpenAI()
reload_conversation = ConversationChain(llm=llm, memory=pickle.loads(new_memory_loaded))

In [38]:
print(reload_conversation.memory.buffer)

Human: Hello, nice to meet you
AI: Hello! It's a pleasure to meet you too. I'm an AI designed to assist with various tasks and provide information. How can I help you today?
Human: Tell me about an interesting Physics fact.
AI: Sure! Here's an interesting Physics fact for you: Did you know that light travels at approximately 299,792,458 meters per second in a vacuum? This is known as the speed of light, and it is the fastest speed that anything can travel in the universe. It plays a crucial role in various phenomena, such as determining the behavior of electromagnetic waves and the principles of relativity.


# ConversationBufferWindowMemory
Limited window size (interactions)

In [39]:
from langchain.memory import ConversationBufferWindowMemory

In [40]:
# Human and AI is one interaction
# k=1 WINDOW --> k interactions

In [42]:
llm = ChatOpenAI()
memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(llm=llm, memory=memory)

In [43]:
conversation.predict(input="Hello, how are you?")

"Hello! I'm an AI, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"

In [44]:
conversation.predict(input="Tell me a math fact.")

'Sure! Did you know that the number pi (π) is an irrational number, which means it cannot be expressed as a simple fraction? Its decimal representation goes on forever without repeating, making it a fascinating and mysterious number in mathematics.'

In [45]:
conversation.predict(input="Tell me a fact about mars.")

'Certainly! Mars is the fourth planet from the Sun in our solar system. It is often called the "Red Planet" due to its reddish appearance, which is caused by iron oxide (or rust) on its surface. Mars has a thin atmosphere composed mostly of carbon dioxide, which makes it inhospitable to human life as we know it. However, scientists believe that Mars may have had liquid water in the past, and there is ongoing research to determine if Mars could support microbial life.'

In [49]:
print(conversation.memory.buffer)

Human: Tell me a fact about mars.
AI: Certainly! Mars is the fourth planet from the Sun in our solar system. It is often called the "Red Planet" due to its reddish appearance, which is caused by iron oxide (or rust) on its surface. Mars has a thin atmosphere composed mostly of carbon dioxide, which makes it inhospitable to human life as we know it. However, scientists believe that Mars may have had liquid water in the past, and there is ongoing research to determine if Mars could support microbial life.


# ConversationSummaryMemory

In [50]:
from langchain.memory import ConversationSummaryBufferMemory

In [57]:
llm = ChatOpenAI()

# we need to parse a model because it will use it to generate a summary
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) 

# Human AI Human AI --> LLM Summary --> 

In [58]:
conversation = ConversationChain(llm=llm, memory=memory)

In [59]:
conversation.predict(input="Give me some travel plans for San Francisco for January 2024")

"Sure! San Francisco is a beautiful city with lots to offer. In January 2024, there are several exciting things you can do. Here are some travel plans for you:\n\n1. Visit the Golden Gate Bridge: This iconic landmark is a must-see in San Francisco. You can take a walk across the bridge and enjoy breathtaking views of the city and the bay.\n\n2. Explore Alcatraz Island: Take a ferry ride to Alcatraz Island and tour the famous former prison. Learn about its intriguing history and enjoy stunning views of the city skyline.\n\n3. Visit Fisherman's Wharf: This vibrant waterfront area is full of shops, restaurants, and attractions. You can indulge in delicious seafood, go on a bay cruise, or visit the famous Pier 39 with its sea lions.\n\n4. Explore Chinatown: San Francisco's Chinatown is the largest in North America. Take a stroll through its colorful streets, visit temples, and try authentic Chinese cuisine.\n\n5. Explore Golden Gate Park: This expansive park offers a variety of attractions

In [60]:
conversation.predict(input="Give me some travel plans for New York City for December 2024")

"In December 2024, New York City will be bustling with holiday spirit and festive activities. Here are some travel plans for your visit:\n\n1. Visit Rockefeller Center: Enjoy the iconic Christmas tree and go ice skating in the Rockefeller Center ice rink. The area is beautifully decorated with lights and holiday decorations.\n\n2. Explore Times Square: Experience the vibrant energy of Times Square, especially during the New Year's Eve celebrations. You can watch the famous ball drop and enjoy the lively atmosphere.\n\n3. Take a walk in Central Park: Despite the colder weather, Central Park is still a wonderful place to visit. You can take a stroll through the park, enjoy the peaceful surroundings, and maybe even catch a horse-drawn carriage ride.\n\n4. Discover the holiday markets: New York City is known for its diverse holiday markets, such as the Union Square Holiday Market and the Bryant Park Winter Village. These markets offer unique gifts, delicious food, and a festive atmosphere.

In [66]:
conversation.memory.load_memory_variables({})

{'history': "System: The human asks the AI for travel plans for San Francisco in January 2024. The AI provides a comprehensive list of attractions and activities, including visiting the Golden Gate Bridge, exploring Alcatraz Island, visiting Fisherman's Wharf, exploring Chinatown, exploring Golden Gate Park, taking a cable car ride, visiting the Painted Ladies, and enjoying the city's culinary scene. The AI also advises checking for specific events or festivals happening during that time. The human then asks for travel plans for New York City in December 2024. The AI provides a list of activities, such as visiting Rockefeller Center, exploring Times Square during New Year's Eve celebrations, taking a walk in Central Park, discovering holiday markets, visiting the Metropolitan Museum of Art, watching a Broadway show, and indulging in holiday treats. The AI also advises checking for specific events or festivals happening in New York City during that time."}